In [1]:
def desugar(code, func_name, func_args, func_body):
    """
    Replaces all occurences of 
       foo = func_name(func_args) 
    with
       func_body[x->a,y->b]
       foo = [result returned in func_body]    
    """
    # Uses Python regular expressions to simplify the search and replace,
    # see https://docs.python.org/3/library/re.html and Chapter 9 of the book

    # regular expression for capturing a list of variable names separated by commas
    arglist = ",".join([r"([a-zA-Z0-9\_\[\]]+)" for i in range(len(func_args))])
    # regular expression for capturing a statement of the form
    # "variable = func_name(arguments)"
    regexp = fr'([a-zA-Z0-9\_\[\]]+)\s*=\s*{func_name}\({arglist}\)\s*$'
    while True:
        m = re.search(regexp, code, re.MULTILINE)
        if not m: break
        newcode = func_body 
        # replace function arguments by the variables from the function invocation
        for i in range(len(func_args)): 
            newcode = newcode.replace(func_args[i], m.group(i+2))
        # Splice the new code inside
        newcode = newcode.replace('return', m.group(1) + " = ")
        code = code[:m.start()] + newcode + code[m.end()+1:]
    return code

In [2]:
def parse_func(code):
    """Parse a function definition into name, arguments and body"""
    lines = [l.strip() for l in code.split('\n')]
    regexp = r'def\s+([a-zA-Z\_0-9]+)\(([\sa-zA-Z0-9\_,]+)\)\s*:\s*'
    m = re.match(regexp, lines[0])
    return m.group(1), m.group(2).split(','), '\n'.join(lines[1:])

In [4]:
def IF(cond, a, b):
    """NAND-CIRC Implementation"""
    notcond = NAND(cond, cond)
    temp = NAND(b, notcond)
    temp1 = NAND(a, cond)
    return NAND(temp, temp1)

In [6]:
def IF(cond, a, b):
    t1 = AND(cond, a)
    notcond = NOT(cond)
    t2 = AND(notcond, b)
    return OR(t1, t2)

In [11]:
# Add two n-bit integers
# Use LSB first notation for simplicity

n = 5 # number of bits

def ADD(A, B):
    """
    n = fixed integer
    input = 2n bits
    output = n + 1 bits
    """
    Result = [0] * (n+1)
    Carry  = [0] * (n+1)
    Carry[0] = zero(A[0])
    
    for i in range(n):
        Result[i] = XOR(Carry[i], XOR(A[i], B[i]))
        Carry[i+1] = MAJ(Carry[i], A[i], B[i])

    Result[n] = Carry[n]
    return Result

ADD([1, 1, 1, 0, 0], [1, 0, 0, 0, 0]) # [0, 0, 0, 1, 0, 0]

NameError: name 'zero' is not defined